# Estimating local and global feature importance scores using DiCE

Summaries of counterfactual examples can be used to estimate importance of features. Intuitively, a feature that is changed more often to generate a proximal counterfactual is an important feature. We use this intuition to build a feature importance score. 

This score can be interpreted as a measure of the **necessity** of a feature to cause a particular model output. That is, if the feature's value changes, then it is likely that the model's output class will also change (or the model's output will significantly change in case of regression model).  

Below we show how counterfactuals can be used to provide local feature importance scores for any input, and how those scores can be combined to yield a global importance score for each feature.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

## Preliminaries: Loading the data and ML model

In [3]:
from dice_ml.utils import helpers # helper functions
dataset = helpers.load_adult_income_dataset().sample(5000) # downsampling to reduce ML model fitting time
helpers.get_adult_data_info()

{'age': 'age',
 'workclass': 'type of industry (Government, Other/Unknown, Private, Self-Employed)',
 'education': 'education level (Assoc, Bachelors, Doctorate, HS-grad, Masters, Prof-school, School, Some-college)',
 'marital_status': 'marital status (Divorced, Married, Separated, Single, Widowed)',
 'occupation': 'occupation (Blue-Collar, Other/Unknown, Professional, Sales, Service, White-Collar)',
 'race': 'white or other race?',
 'gender': 'male or female?',
 'hours_per_week': 'total work hours per week',
 'income': '0 (<=50K) vs 1 (>50K)'}

In [4]:
target = dataset["income"]
# Split data into train and test
from sklearn.model_selection import train_test_split
datasetX = dataset.drop("income", axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX, 
                                                    target, 
                                                    test_size = 0.2,
                                                    random_state=0,
                                                    stratify=target)

numerical=["age", "hours_per_week"]
categorical = x_train.columns.difference(numerical) 
from sklearn.compose import ColumnTransformer

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        ('cat', categorical_transformer, categorical)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', RandomForestClassifier())])
model = clf.fit(x_train, y_train)

In [5]:
import dice_ml
from dice_ml import Dice

d = dice_ml.Data(dataframe=dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')
m = dice_ml.Model(model=model, backend="sklearn")

## Local feature importance

We first generate counterfactuals for a given input point. 

In [11]:
exp = Dice(d, m, method="random")
query_instance = x_train[1:2]
e1 = exp.generate_counterfactuals(query_instance, total_CFs=10, desired_range=None,desired_class="opposite",
                                   permitted_range=None, features_to_vary="all")
e1.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,49,Private,HS-grad,Divorced,White-Collar,White,Female,40,0



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,-,Married,-,-,-,-,1
1,-,-,Bachelors,-,-,-,Male,-,1
2,-,-,Masters,Married,-,-,-,-,1
3,-,-,-,-,-,-,Male,48.0,1
4,-,-,Masters,-,-,-,-,97.0,1
5,-,Government,-,Married,-,-,-,-,1
6,-,-,Masters,-,-,-,Male,-,1
7,-,-,-,Married,-,-,Male,-,1
8,-,-,Masters,Widowed,-,-,-,-,-
9,-,-,Prof-school,-,-,-,Male,-,1


These can now be used to calculate the feature importance scores. 

In [7]:
imp = exp.local_feature_importance(query_instance, cf_examples_list=e1.cf_examples_list)
print(imp.local_importance)

[{'education': 0.6, 'marital_status': 0.5, 'hours_per_week': 0.3, 'age': 0.2, 'workclass': 0.1, 'occupation': 0.1, 'race': 0.1, 'gender': 0.1}]


Feature importance can also be estimated directly, by leaving the `cf_examples_list` argument blank.

In [8]:
imp = exp.local_feature_importance(query_instance, posthoc_sparsity_param=None)
print(imp.local_importance)

[{'education': 0.6, 'marital_status': 0.6, 'gender': 0.2, 'age': 0.2, 'workclass': 0.1, 'occupation': 0.1, 'race': 0.1, 'hours_per_week': 0.1}]


## Global importance

For global importance, we need to generate counterfactuals for a representative sample of the dataset. 

In [12]:
cobj=exp.global_feature_importance(x_train[0:10], total_CFs=10, posthoc_sparsity_param=None)
print(cobj.summary_importance)

{'age': 0.57, 'marital_status': 0.4, 'education': 0.37, 'occupation': 0.27, 'hours_per_week': 0.23, 'race': 0.16, 'workclass': 0.13, 'gender': 0.13}


## Convert the counterfactual output to json

In [13]:
json_str = cobj.to_json()
print(json_str)

{
  "cf_examples_list": [
    "{\"data_interface\": {\"outcome_name\": \"income\", \"data_df\": \"dummy_data\"}, \"model_type\": \"classifier\", \"desired_class\": \"opposite\", \"desired_range\": null, \"test_instance_df\": \"{\\\"age\\\":{\\\"0\\\":35},\\\"workclass\\\":{\\\"0\\\":\\\"Private\\\"},\\\"education\\\":{\\\"0\\\":\\\"HS-grad\\\"},\\\"marital_status\\\":{\\\"0\\\":\\\"Married\\\"},\\\"occupation\\\":{\\\"0\\\":\\\"Blue-Collar\\\"},\\\"race\\\":{\\\"0\\\":\\\"White\\\"},\\\"gender\\\":{\\\"0\\\":\\\"Male\\\"},\\\"hours_per_week\\\":{\\\"0\\\":40},\\\"income\\\":{\\\"0\\\":0}}\", \"final_cfs_df\": \"{\\\"age\\\":{\\\"0\\\":50.0,\\\"1\\\":35,\\\"2\\\":59.0,\\\"3\\\":48.0,\\\"4\\\":56.0,\\\"5\\\":59.0,\\\"6\\\":35,\\\"7\\\":35,\\\"8\\\":55.0,\\\"9\\\":35},\\\"workclass\\\":{\\\"0\\\":\\\"Private\\\",\\\"1\\\":\\\"Private\\\",\\\"2\\\":\\\"Private\\\",\\\"3\\\":\\\"Private\\\",\\\"4\\\":\\\"Private\\\",\\\"5\\\":\\\"Private\\\",\\\"6\\\":\\\"Private\\\",\\\"7\\\":\\\"Private\\

## Convert the json output to a counterfactual object

In [14]:
imp_r=imp.from_json(json_str)
print([o.visualize_as_dataframe(show_only_changes=True) for o in imp_r.cf_examples_list])
print(imp_r.local_importance)
print(imp_r.summary_importance)

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,35,Private,HS-grad,Married,Blue-Collar,White,Male,40,0



Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,50,-,-,-,-,-,-,-,1
1,-,-,-,-,Professional,Other,-,-,1
2,59,-,-,-,White-Collar,-,-,-,1
3,48,-,-,-,Other/Unknown,-,-,-,1
4,56,-,-,-,-,Other,-,-,1
5,59,-,-,-,-,-,-,-,1
6,-,-,-,-,Professional,-,-,-,1
7,-,-,Masters,-,Sales,-,-,-,1
8,55,-,-,-,-,-,-,-,1
9,-,Government,-,-,-,Other,-,-,1


Query instance (original outcome : 0)


...

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,20,Private,HS-grad,Single,Service,White,Female,40,0



Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,78,Self-Employed,-,Married,-,-,-,-,1
1,45,Government,-,Married,Blue-Collar,-,-,36,1
2,73,-,Doctorate,-,White-Collar,-,Male,-,1
3,79,-,Prof-school,Married,-,-,Male,60,1
4,63,Government,-,Married,-,Other,-,-,1
5,63,Government,-,Married,Other/Unknown,-,-,-,1
6,49,-,-,Married,-,Other,-,-,1
7,49,-,-,Married,Sales,Other,Male,-,1
8,52,-,-,Married,-,-,Male,-,1
9,85,Government,Prof-school,-,-,-,Male,-,-


[None, None, None, None, None, None, None, None, None, None]
[{'age': 0.6, 'occupation': 0.5, 'race': 0.3, 'workclass': 0.1, 'education': 0.1, 'marital_status': 0.0, 'gender': 0.0, 'hours_per_week': 0.0}, {'marital_status': 0.7, 'education': 0.6, 'gender': 0.2, 'age': 0.2, 'occupation': 0.1, 'race': 0.1, 'hours_per_week': 0.1, 'workclass': 0.0}, {'education': 0.4, 'age': 0.4, 'occupation': 0.3, 'race': 0.3, 'marital_status': 0.2, 'hours_per_week': 0.2, 'workclass': 0.1, 'gender': 0.0}, {'hours_per_week': 0.7, 'age': 0.4, 'marital_status': 0.2, 'workclass': 0.1, 'occupation': 0.1, 'education': 0.0, 'race': 0.0, 'gender': 0.0}, {'age': 0.6, 'marital_status': 0.3, 'education': 0.2, 'occupation': 0.2, 'hours_per_week': 0.2, 'gender': 0.1, 'workclass': 0.0, 'race': 0.0}, {'age': 0.5, 'education': 0.3, 'race': 0.3, 'occupation': 0.2, 'hours_per_week': 0.2, 'workclass': 0.1, 'gender': 0.1, 'marital_status': 0.0}, {'marital_status': 0.4, 'occupation': 0.4, 'hours_per_week': 0.4, 'education': 0